## 1. Setup：環境設定

首先載入必要的套件，並設定顯示選項與中文字型。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 設定中文字型（避免圖表中文顯示為方塊）
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定 pandas 顯示選項
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ 套件載入完成！")

## 2. 載入資料

讀取電商銷售資料檔案 `ecommerce_sales.csv`。

In [ ]:
# 讀取 CSV 檔案
df = pd.read_csv('ecommerce_sales.csv')

# 確保日期欄位為 datetime 格式
df['order_date'] = pd.to_datetime(df['order_date'])

print(f"✓ 資料載入完成！共 {len(df)} 筆訂單")

## 3. 資料概覽

在進行分析前，先全面了解資料的結構、型別與統計特性。

### 3.1 查看前幾筆資料

In [ ]:
# 顯示前 5 筆資料
df.head()

### 3.2 資料大小與欄位

In [ ]:
# 查看資料大小（列數、欄數）
print(f"資料大小：{df.shape}")
print(f"共有 {df.shape[0]} 筆訂單，{df.shape[1]} 個欄位")
print()

# 查看欄位名稱
print("欄位名稱：")
print(df.columns.tolist())

### 3.3 資料型別與缺失值

In [ ]:
# 查看每個欄位的資料型別與缺失值
df.info()

### 3.4 數值欄位統計摘要

In [ ]:
# 顯示數值欄位的統計資訊
df.describe()

## 4. 特徵工程

根據現有欄位，計算出新的商業指標：營收與利潤。

In [ ]:
# 計算營收（revenue）= 單價 × 數量
df['revenue'] = df['price'] * df['quantity']

# 計算利潤（profit）= (單價 - 成本) × 數量
df['profit'] = (df['price'] - df['cost']) * df['quantity']

print("✓ 特徵工程完成！")
print()

# 檢視新增的欄位
df[['order_id', 'price', 'cost', 'quantity', 'revenue', 'profit']].head()

## 5. 整體業務指標

計算整體營運表現的關鍵指標。

In [ ]:
# 計算總營收
total_revenue = df['revenue'].sum()

# 計算總利潤
total_profit = df['profit'].sum()

# 計算平均訂單金額（AOV - Average Order Value）
average_order_value = df['revenue'].mean()

# 計算總訂單數
total_orders = len(df)

print("=== 整體業務表現 ===")
print(f"總訂單數：{total_orders:,} 筆")
print(f"總營收：${total_revenue:,.2f}")
print(f"總利潤：${total_profit:,.2f}")
print(f"平均訂單金額：${average_order_value:.2f}")
print(f"利潤率：{(total_profit / total_revenue * 100):.2f}%")

## 6. 產品與類別分析

分析不同產品類別與個別產品的銷售表現。

### 6.1 依類別分析

In [ ]:
# 計算各類別的營收、利潤與訂單數
category_stats = df.groupby('category').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'order_id': 'count'
}).round(2)

# 重新命名欄位
category_stats.columns = ['總營收', '總利潤', '訂單數']

# 依總營收排序
category_stats = category_stats.sort_values('總營收', ascending=False)

print("=== 各類別業績表現 ===")
category_stats

### 6.2 依產品分析

In [ ]:
# 計算各產品的營收與銷售數量
product_stats = df.groupby('product').agg({
    'revenue': 'sum',
    'quantity': 'sum',
    'order_id': 'count'
}).round(2)

product_stats.columns = ['總營收', '銷售數量', '訂單數']
product_stats = product_stats.sort_values('總營收', ascending=False)

print("=== 各產品業績表現（前 10 名）===")
product_stats.head(10)

## 7. 通路與地區分析

探索不同銷售通路與地區的表現差異。

### 7.1 依銷售通路分析

In [ ]:
# 各通路的營收與訂單數
channel_stats = df.groupby('channel').agg({
    'revenue': ['sum', 'mean'],
    'order_id': 'count'
}).round(2)

channel_stats.columns = ['總營收', '平均訂單金額', '訂單數']
channel_stats = channel_stats.sort_values('總營收', ascending=False)

print("=== 各通路表現 ===")
channel_stats

### 7.2 依地區分析

In [ ]:
# 各地區的營收與訂單數
region_stats = df.groupby('region').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'order_id': 'count'
}).round(2)

region_stats.columns = ['總營收', '總利潤', '訂單數']
region_stats = region_stats.sort_values('總營收', ascending=False)

print("=== 各地區表現 ===")
region_stats

### 7.3 地區與通路交叉分析

In [ ]:
# 建立地區 × 通路的營收透視表
pivot_table = df.pivot_table(
    values='revenue',
    index='region',
    columns='channel',
    aggfunc='sum'
).round(2)

print("=== 地區 × 通路 營收表 ===")
pivot_table

## 8. 營收集中度分析

分析高價值訂單對整體營收的貢獻度。

In [ ]:
# 依營收排序訂單
df_sorted = df.sort_values('revenue', ascending=False).reset_index(drop=True)

# 計算前 10% 訂單的營收貢獻
top_10_percent_count = int(len(df) * 0.1)
top_10_percent_revenue = df_sorted.head(top_10_percent_count)['revenue'].sum()
contribution_rate = (top_10_percent_revenue / total_revenue) * 100

print("=== 營收集中度分析 ===")
print(f"總訂單數：{len(df):,} 筆")
print(f"前 10% 訂單數：{top_10_percent_count:,} 筆")
print(f"前 10% 訂單總營收：${top_10_percent_revenue:,.2f}")
print(f"貢獻率：{contribution_rate:.2f}%")
print()
print("→ 這顯示高價值訂單對整體營收的重要性")

## 9. 時間序列分析

觀察營收隨時間的變化趨勢。

### 9.1 每日營收

In [ ]:
# 計算每日總營收
daily_revenue = df.groupby('order_date')['revenue'].sum().reset_index()
daily_revenue.columns = ['日期', '營收']

print("=== 每日營收統計 ===")
print(f"資料期間：{daily_revenue['日期'].min()} 至 {daily_revenue['日期'].max()}")
print(f"平均每日營收：${daily_revenue['營收'].mean():,.2f}")
print(f"最高單日營收：${daily_revenue['營收'].max():,.2f}")
print(f"最低單日營收：${daily_revenue['營收'].min():,.2f}")
print()

# 顯示前 10 天
daily_revenue.head(10)

### 9.2 每週營收

In [ ]:
# 將日期轉換為週（設定週一為第一天）
df['week'] = df['order_date'].dt.to_period('W-MON')

# 計算每週總營收
weekly_revenue = df.groupby('week')['revenue'].sum().reset_index()
weekly_revenue.columns = ['週別', '營收']

print("=== 每週營收統計 ===")
print(f"平均每週營收：${weekly_revenue['營收'].mean():,.2f}")
print()

weekly_revenue.head(10)

## 10. 資料視覺化

使用圖表呈現關鍵發現，讓數據更容易理解。

### 10.1 營收分布直方圖

In [ ]:
# 繪製營收分布直方圖
plt.figure(figsize=(12, 6))
plt.hist(df['revenue'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
plt.title('訂單營收分布', fontsize=16, fontweight='bold')
plt.xlabel('營收金額', fontsize=12)
plt.ylabel('訂單數量', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 10.2 各類別營收長條圖

In [ ]:
# 繪製各類別營收長條圖
plt.figure(figsize=(10, 6))
category_revenue = df.groupby('category')['revenue'].sum().sort_values(ascending=False)
category_revenue.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('各類別總營收', fontsize=16, fontweight='bold')
plt.xlabel('產品類別', fontsize=12)
plt.ylabel('總營收', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 10.3 每日營收趨勢折線圖

In [ ]:
# 繪製每日營收趨勢
plt.figure(figsize=(14, 6))
plt.plot(daily_revenue['日期'], daily_revenue['營收'], 
         linewidth=2, color='green', marker='o', markersize=3)
plt.title('每日營收趨勢', fontsize=16, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('營收', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. 總結與商業洞察

透過以上分析，我們獲得以下重要發現：

### 📊 關鍵發現

**1. 整體業務表現**
- 我們分析了數千筆訂單，計算出總營收、總利潤與平均訂單金額
- 利潤率顯示了業務的獲利能力

**2. 產品與類別洞察**
- 不同產品類別的營收貢獻差異明顯
- 某些類別雖然訂單數較少，但平均訂單金額較高
- 可根據表現調整產品策略與庫存管理

**3. 通路與地區策略**
- 不同銷售通路（線上/實體）各有優勢
- 地區表現差異可指引行銷資源分配
- 透視表揭示了地區與通路的組合效應

**4. 營收集中度**
- 前 10% 的高價值訂單貢獻了相當比例的總營收
- 顯示維護高價值客戶的重要性

**5. 時間趨勢**
- 每日營收變化反映了市場波動
- 週營收統計幫助識別長期趨勢
- 可用於預測未來銷售並制定目標

### 🎯 商業建議

1. **產品策略**：聚焦高營收類別，優化低表現產品
2. **客戶管理**：建立 VIP 客戶計畫，維護高價值訂單
3. **通路優化**：根據表現分配行銷預算
4. **地區擴展**：在高潛力地區加強投資
5. **庫存管理**：根據銷售數據調整庫存策略

### 📈 下一步分析方向

- 客戶分群分析（RFM 模型）
- 產品關聯分析（購物籃分析）
- 季節性與週期性分析
- 預測模型建立

---

**恭喜完成電商數據分析專案！** 🎉

你已經掌握了：
- ✅ 完整的數據分析流程
- ✅ 特徵工程技巧
- ✅ Groupby 與 Pivot Table 應用
- ✅ 時間序列分析
- ✅ 資料視覺化
- ✅ 商業洞察提煉

繼續實踐，將數據轉化為商業價值！💪